## Estimación Ku Omega
Codigo generado por ChatGPT

In [6]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

def calcular_parametros(df):
    # Medir el valor pico a pico de la salida y(t)
    ypp = df["y(t)"].max() - df["y(t)"].min()
    a = ypp / 2
    
    # Encontrar el periodo Tu de la oscilación de u(t)
    difs = np.diff(df["u(t)"].values)
    cruces = np.where(np.sign(difs[:-1]) != np.sign(difs[1:]))[0]
    if len(cruces) > 1:
        Tu = np.mean(np.diff(df["tiempo"].iloc[cruces]))
    else:
        Tu = np.nan  # No se detectaron suficientes cruces
    
    # Calcular Ku y ωu
    e = 0  # Ajuste según el sistema
    Ku = (4 / np.pi) * (df["u(t)"].max() / np.sqrt(a**2 - e**2)) if a > e else np.nan
    omega_u = (2 * np.pi) / Tu if Tu > 0 else np.nan
    
    return Ku, omega_u

def graficar_datos(archivo, n_skip):
    # Cargar datos omitiendo las primeras n filas
    df = pd.read_csv(archivo, skiprows=n_skip, sep=",", engine="python")
    
    # Nombrar columnas
    df.columns = ["tiempo", "u(t)", "y(t)"]
    
    # Calcular parámetros del sistema
    Ku, omega_u = calcular_parametros(df)
    
    # Crear gráfica con zoom habilitado
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["tiempo"], y=df["u(t)"], mode='lines+markers', name="u(t)"))
    fig.add_trace(go.Scatter(x=df["tiempo"], y=df["y(t)"], mode='lines+markers', name="y(t)"))
    
    fig.update_layout(title=f"Gráfica de u(t) y y(t)\nKu={Ku:.3f}, ωu={omega_u:.3f}",
                      xaxis_title="Tiempo (s)",
                      yaxis_title="Valores",
                      template="plotly_white",
                      xaxis=dict(rangemode='normal'),
                      yaxis=dict(rangemode='normal'))
    
    fig.show()

# Graficar por primera vez
graficar_datos("salida.txt", n_skip=8500)
